In [2]:
from langchain_community.document_loaders import PDFMinerLoader
import csv
import re
import pandas as pd

In [2]:
# from langchain_community.document_loaders import DirectoryLoader

# loader = DirectoryLoader('../data/contracts', glob="*.pdf", loader_cls=PDFMinerLoader)
# docs = loader.load()

In [3]:
loader = PDFMinerLoader('../data/contracts/Robinson Advisory.docx.pdf')
docs = loader.load()

In [4]:
len(docs[0].page_content)

14343

In [5]:
qna = pd.read_csv('../data/Q&A/Robinson Q&A - Sheet1.csv')
qna

,Questions,Answers
0,Who are the parties to the Agreement and what ...,Cloud Investments Ltd. (“Company”) and Jack Ro...
1,What is the termination notice?,According to section 4:14 days for convenience...
2,What are the payments to the Advisor under th...,According to section 6: 1. Fees of $9 per hour...
3,Can the Agreement or any of its obligations b...,1. Under section 1.1 the Advisor can’t assign ...
4,Who owns the IP?,According to section 4 of the Undertaking (App...
5,Is there a non-compete obligation to the Advisor?,Yes. During the term of engagement with the Co...
6,Can the Advisor charge for meal time?,"No. See Section 6.1, Billable Hour doesn’t inc..."
7,In which street does the Advisor live?,"1 Rabin st, Tel Aviv, Israel"
8,Is the Advisor entitled to social benefits?,"No. According to section 8 of the Agreement, t..."
9,What happens if the Advisor claims compensatio...,If the Advisor is determined to be an employe...


In [6]:
import nest_asyncio
import os
import openai
from dotenv import load_dotenv

load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")

openai.api_key = api_key

nest_asyncio.apply()

In [7]:
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

index = VectorstoreIndexCreator().from_loaders([loader])

llm = ChatOpenAI(temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=index.vectorstore.as_retriever(),
    return_source_documents=True,
)

/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
eval_questions = qna['Questions'].tolist()
eval_answers = qna['Answers'].tolist()

In [9]:
examples  = [
    {
        "query": q, "ground_truths": [eval_answers[i]]
    }
    for i, q in enumerate(eval_questions)
]

In [12]:
from ragas.langchain.evalchain import RagasEvaluatorChain
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)

faithfulness_chain = RagasEvaluatorChain(metric=faithfulness)
answer_rel_chain = RagasEvaluatorChain(metric=answer_relevancy)
context_rel_chain = RagasEvaluatorChain(metric=context_precision)
context_recall_chain = RagasEvaluatorChain(metric=context_recall)

In [14]:
predictions = qa_chain.batch(examples)

In [16]:
faithfulness_score = faithfulness_chain.evaluate(examples, predictions)
answer_relevancy_score = answer_rel_chain.evaluate(examples, predictions)
context_precision_score = context_rel_chain.evaluate(examples, predictions)
context_recall_score = context_recall_chain.evaluate(examples, predictions)

100%|██████████| 2/2 [00:43<00:00, 21.57s/it]
/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)
100%|██████████| 2/2 [00:28<00:00, 14.49s/it]
/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/datasets/table.py:1401: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/home/misge/miniconda3/envs/wk6/lib/python3.11/site-packages/datasets/table.py:1427: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)
100%|██████████| 2/2 [00:12<00:00,  6.26s/it]
/home/misge/minicond

NameError: name 'context_recall_chan' is not defined

In [24]:
faithfulness_scores = [score["faithfulness_score"] for score in faithfulness_score]
answer_relevancy_scores = [score["answer_relevancy_score"] for score in answer_relevancy_score]
context_precision_scores = [score["context_precision_score"] for score in context_precision_score]
context_recall_scores = [score["context_recall_score"] for score in context_recall_score]

In [46]:
df = pd.DataFrame({
        "Faithfulness Score": faithfulness_scores,
        "Answer Relevancy Score": answer_relevancy_scores,
        "Context Precision Score": context_precision_scores
    })

In [47]:
df

,Faithfulness Score,Answer Relevancy Score,Context Precision Score
0,0.666667,0.983827,0.000000
1,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000
3,1.000000,0.979638,0.500000
4,1.000000,0.933162,0.000000
5,1.000000,0.983585,1.000000
6,1.000000,0.931222,1.000000
7,1.000000,0.931554,1.000000
8,0.500000,0.932479,0.000000
9,1.000000,0.965923,0.500000


In [48]:
df2 = pd.DataFrame(examples)

In [49]:
df2

,query,ground_truths
0,Under what circumstances and to what extent th...,"[Except in the case of fraud, the Sellers have..."
1,How much is the escrow amount?,"[The escrow amount is equal to $1,000,000.]"
2,Is escrow amount grete then the Retention Amou...,[No]
3,What is the purpose of the escrow?,[To serve as a recourse of the Buyer in case o...
4,Are there any conditions to the closing?,"[No, as the signing and closing are simultaneo..."
5,Are Change of Control Payments considered a Se...,[Yes. (See defining of Sellers Transaction Exp...
6,Would the aggregate amount payable by the Buye...,[Yes (See Section 2.07)]
7,Does the Buyer need to pay the Employees Closi...,[No.]
8,Does any of the Sellers provide a representati...,[No. Only the Company provides such a represen...
9,Is any of the Sellers bound by a non-competiti...,[No.]


In [54]:
new_df = pd.concat([df2, df], axis=1)

In [55]:
new_df

,query,ground_truths,Faithfulness Score,Answer Relevancy Score,Context Precision Score
0,Under what circumstances and to what extent th...,"[Except in the case of fraud, the Sellers have...",0.666667,0.983827,0.000000
1,How much is the escrow amount?,"[The escrow amount is equal to $1,000,000.]",0.000000,0.000000,0.000000
2,Is escrow amount grete then the Retention Amou...,[No],0.000000,0.000000,0.000000
3,What is the purpose of the escrow?,[To serve as a recourse of the Buyer in case o...,1.000000,0.979638,0.500000
4,Are there any conditions to the closing?,"[No, as the signing and closing are simultaneo...",1.000000,0.933162,0.000000
5,Are Change of Control Payments considered a Se...,[Yes. (See defining of Sellers Transaction Exp...,1.000000,0.983585,1.000000
6,Would the aggregate amount payable by the Buye...,[Yes (See Section 2.07)],1.000000,0.931222,1.000000
7,Does the Buyer need to pay the Employees Closi...,[No.],1.000000,0.931554,1.000000
8,Does any of the Sellers provide a representati...,[No. Only the Company provides such a represen...,0.500000,0.932479,0.000000
9,Is any of the Sellers bound by a non-competiti...,[No.],1.000000,0.965923,0.500000
